In [1]:
# №1

iter = 100 # кол-во тераций
x_old = 0
a = 0.1 # шаг сходиомсти
ee = 1e-4
         
for i in range(iter):
    x_new = x_old - a * (2*x_old + 4)
    
    if abs(x_new - x_old) < ee:
        break
    
    x_old = x_new
    
min_value = x_old**2 + 4*x_old + 4

print(round(x_old), round(min_value))

-2 0


In [1]:
# №2.1

# Импортируем функцию linprog из модуля optimize библиотеки SciPy
from scipy.optimize import linprog
# Коэффициенты целевой функции (для минимизации, поэтому отрицательные)
c = [-5, -3]
# Матрица коэффициентов ограничений (Ax <= b)
A = [[2, 1]]
# Вектор правых частей ограничений
b = [100]
# Границы для переменных (x >= 0, y >= 0)
x_bounds = (0, None)
y_bounds = (0, None)
# Решаем задачу линейного программирования
result = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds], method='simplex')
# Проверяем успешность решения
if result.success:
    # Выводим результаты
    print(f"Оптимальное количество товара A: {result.x[0]:.2f}")
    print(f"Оптимальное количество товара B: {result.x[1]:.2f}")
    print(f"Максимальная прибыль: {-result.fun:.2f}")
else:
    print("Задача не имеет решения или не удалось найти оптимальное решение.")

Оптимальное количество товара A: 0.00
Оптимальное количество товара B: 100.00
Максимальная прибыль: 300.00


C:\Users\kache\AppData\Local\Temp\ipykernel_8600\2530816124.py:15: DeprecationWarning: `method='simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  result = linprog(c, A_ub=A, b_ub=b, bounds=[x_bounds, y_bounds], method='simplex')


In [2]:
# №2.2

import numpy as np

def simplex(c, A, b):
    # Количество ограничений и переменных
    m, n = A.shape

    # Добавляем искусственные переменные (в данном случае одна)
    c_extended = np.append(c, np.zeros(m))
    A_extended = np.hstack((A, np.eye(m)))

    # Создаем таблицу симплекс-метода
    table = np.zeros((m + 1, n + m + 1))
    table[:-1, :-1] = A_extended
    table[:-1, -1] = b
    table[-1, :-1] = -c_extended

    # Итерационный процесс
    while True:
        # Проверка на оптимальность
        if all(table[-1, :-1] >= 0):
            break

        # Выбор входящей переменной (максимальный коэффициент в Z-строке)
        pivot_col = np.argmax(table[-1, :-1])

        # Выбор исходящей переменной
        ratios = []
        for i in range(m):
            if table[i, pivot_col] > 0:
                ratio = table[i, -1] / table[i, pivot_col]
                ratios.append((i, ratio))
            else:
                ratios.append((i, float('inf')))

        # Находим минимальное отношение
        pivot_row, _ = min(ratios, key=lambda x: (x[1], x[0]))

        # Пересчет таблицы
        pivot_element = table[pivot_row, pivot_col]
        table[pivot_row, :] /= pivot_element

        for i in range(m + 1):
            if i != pivot_row:
                factor = table[i, pivot_col]
                table[i, :] -= factor * table[pivot_row, :]

    # Извлекаем решение
    solution = np.zeros(n)
    for i in range(m):
        # Находим базисные переменные
        basis = np.where(table[i, :n] == 1)[0]
        if len(basis) == 1:
            var = basis[0]
            solution[var] = table[i, -1]

    # Вычисляем прибыль
    max_profit = np.dot(c, solution)

    return solution, max_profit

# Параметры задачи
# Целевая функция: max Z = 5A + 3B → для симплекса используем [-5, -3] (минимизируем -Z)
c = np.array([-5, -3])

# Матрица ограничений (2A + B ≤ 100)
A = np.array([[2, 1]])

# Правая часть ограничения
b = np.array([100])

# Решение
solution, max_profit = simplex(c, A, b)

print(f"Оптимальное количество товара A: {solution[0]:.0f}")
print(f"Оптимальное количество товара B: {solution[1]:.0f}")
print(f"Максимальная прибыль: {max_profit * -1:.0f}")  # Умножаем на -1 для восстановления максимума

Оптимальное количество товара A: 0
Оптимальное количество товара B: 100
Максимальная прибыль: 300


In [3]:
# №3


def knapsack(items, capacity):
    n = len(items)
    max_value = 0
    best_subset = []

    # Перебираем все возможные маски (подмножества)
    for mask in range(0, 1 << n):  # 1 << n = 2^n
        current_weight = 0
        current_value = 0
        subset = []

        for i in range(n):
            if (mask >> i) & 1:  # Проверяем, включён ли i-й предмет
                weight, value = items[i]
                current_weight += weight
                current_value += value
                subset.append(items[i])

        # Проверяем, не превышает ли вес ограничение и обновляем максимум
        if current_weight <= capacity and current_value > max_value:
            max_value = current_value
            best_subset = subset

    return best_subset, max_value

# Пример использования
items = [(10, 60), (20, 100), (30, 120)]
capacity = 50

selected, total = knapsack(items, capacity)

print("Оптимальный набор предметов:", selected)
print("Максимальная ценность:", total)

Оптимальный набор предметов: [(20, 100), (30, 120)]
Максимальная ценность: 220


In [4]:
# №4


import itertools

def traveling_salesman(graph):
    n = len(graph)
    cities = list(range(n))
    min_length = float('inf')
    best_path = []

    # Начинаем с города 0
    start = 0
    # Перебираем все перестановки остальных городов
    for perm in itertools.permutations(cities[1:]):
        current_path = [start] + list(perm) + [start]
        current_length = 0
        valid = True

        for i in range(len(current_path) - 1):
            a = current_path[i]
            b = current_path[i+1]
            if graph[a][b] == 0:  # Проверка на связность (если путь недоступен)
                valid = False
                break
            current_length += graph[a][b]

        if valid and current_length < min_length:
            min_length = current_length
            best_path = current_path

    return best_path, min_length

# Пример использования
graph = [
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
]

path, length = traveling_salesman(graph)

print("Минимальная длина пути:", length)
print("Порядок посещения городов:", path)

Минимальная длина пути: 80
Порядок посещения городов: [0, 1, 3, 2, 0]


In [ ]:
# №5

import sympy as sp

# Определение переменных
t = sp.symbols('t')
Q0 = 100  # Начальный дебит
lambda_ = 0.05  # Коэффициент падения
T = 365  # Время в днях

# Функция дебита
Q = Q0 * sp.exp(-lambda_ * t)

# Вычисление интеграла от 0 до T
integral = sp.integrate(Q, (t, 0, T))

# Вывод результата
print(f"Накопленная добыча: {integral.evalf():.2f} м³")

Накопленная добыча: 2000.00 м³
